In [ ]:
import torch

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

!nvidia-smi


Torch: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
Sun Jan 18 15:42:54 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  

In [ ]:
# Colab-ready: GPU LLM incident analysis over /content/precomputed_rag.json
# Changes requested:
# 1) Keep the "Issue" as a single, plain operational summary (no comparison text).
# 2) Add "Prev sequence: <ID> | <Classification>" info inside the Reason paragraph.
# 3) Identify which retrieved sequences are the same classification (Normal/Anomalous) when IDs are discoverable.
# 4) Keep a clear, actionable "Suggested fix" paragraph.
# 5) Batch inference for speed; print each sequence result immediately after each batch is generated.

!pip -q install -U transformers accelerate bitsandbytes

import json
import re
from typing import Any, Dict, List, Tuple, Optional

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# ----------------------------
# 1) Config
# ----------------------------
JSON_PATH = "/content/precomputed_rag.json"
MODEL_ID  = "mistralai/Mistral-7B-Instruct-v0.2"

BATCH_SIZE = 6            # Increase if you have VRAM; decrease if OOM
MAX_NEW_TOKENS = 700
TEMPERATURE = 0.2
TOP_P = 0.9
MAX_INPUT_LEN = 4096      # Truncate long logs if needed for speed/stability

# ----------------------------
# 2) Load model (GPU, 4-bit)
# ----------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",         # <-- uses GPU if available
    torch_dtype=torch.float16,
    load_in_4bit=True,         # <-- fits in Colab GPU RAM
)

gen_cfg = GenerationConfig(
    max_new_tokens=MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    do_sample=True,
    repetition_penalty=1.05,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

# Optional quick check (uncomment if you want)
# print("CUDA available:", torch.cuda.is_available())
# if torch.cuda.is_available():
#     print("GPU:", torch.cuda.get_device_name(0))

# ----------------------------
# 3) Helpers: robust extraction and normalization
# ----------------------------
def _first_present(d: Dict[str, Any], keys: List[str], default=None):
    for k in keys:
        if k in d and d[k] is not None:
            return d[k]
    return default

def normalize_classification(x: Any) -> str:
    if x is None:
        return "Unknown"
    s = str(x).strip()
    if s.lower() in ["anomaly", "anomalous", "abnormal", "1", "true"]:
        return "Anomalous"
    if s.lower() in ["normal", "0", "false"]:
        return "Normal"
    return s[:1].upper() + s[1:] if s else "Unknown"

def stringify_logs(logs: Any) -> str:
    if logs is None:
        return ""
    if isinstance(logs, str):
        return logs.strip()
    if isinstance(logs, list):
        parts = []
        for item in logs:
            parts.append(item.strip() if isinstance(item, str) else str(item).strip())
        return "\n".join([p for p in parts if p])
    return str(logs).strip()

def stringify_rag(rag: Any) -> str:
    if rag is None:
        return ""
    if isinstance(rag, str):
        return rag.strip()
    if isinstance(rag, list):
        out = []
        for i, item in enumerate(rag, start=1):
            if isinstance(item, str):
                s = item.strip()
            elif isinstance(item, dict):
                s = _first_present(item, ["text", "log", "logs", "content", "retrieved"], default=str(item)).strip()
            else:
                s = str(item).strip()
            if s:
                out.append(f"[{i}] {s}")
        return "\n".join(out)
    return str(rag).strip()

def stringify_anom_tokens(tokens: Any) -> str:
    if tokens is None:
        return ""
    if isinstance(tokens, str):
        return tokens.strip()

    pairs: List[Tuple[str, float]] = []
    try:
        if isinstance(tokens, dict):
            for k, v in tokens.items():
                try:
                    pairs.append((str(k), float(v)))
                except Exception:
                    pairs.append((str(k), 0.0))
        elif isinstance(tokens, list):
            for item in tokens:
                if isinstance(item, dict):
                    tok = _first_present(item, ["token", "text", "term", "value"], default="")
                    score = _first_present(item, ["score", "weight", "anomaly_score"], default=0.0)
                    try:
                        score = float(score)
                    except Exception:
                        score = 0.0
                    pairs.append((str(tok), score))
                elif isinstance(item, (list, tuple)) and len(item) >= 2:
                    tok, score = item[0], item[1]
                    try:
                        score = float(score)
                    except Exception:
                        score = 0.0
                    pairs.append((str(tok), score))
                else:
                    pairs.append((str(item), 0.0))
    except Exception:
        return str(tokens).strip()

    pairs = sorted(pairs, key=lambda x: x[1], reverse=True)[:8]
    return "\n".join([f"  {t} → {s:.4f}" for t, s in pairs if str(t).strip()])

def get_entries_container(data: Any) -> List[Any]:
    if isinstance(data, list):
        return data
    if isinstance(data, dict):
        entries = _first_present(data, ["entries", "data", "items", "records"], default=None)
        if entries is None:
            # fallback: treat dict values as entries
            return list(data.values())
        return entries
    raise ValueError("Unsupported JSON format: expected list or dict.")

def extract_sequence(entry: Dict[str, Any]) -> Dict[str, str]:
    seq_id = _first_present(entry, ["sequence_id", "sequenceId", "seq_id", "seqId", "id", "Sequence ID"], default="Unknown")
    label  = _first_present(entry, ["classification", "label", "class", "y", "Classification"], default="Unknown")
    logs = _first_present(entry, ["raw_logs", "logs", "log", "raw", "text"], default="")
    rag  = _first_present(entry, ["retrieved_similar_logs", "retrieved", "rag", "RAG", "neighbors"], default="")
    toks = _first_present(entry, ["top_anomalous_tokens", "anomalous_tokens", "tokens", "top_tokens"], default="")

    return {
        "sequence_id": str(seq_id),
        "classification": normalize_classification(label),
        "logs": stringify_logs(logs),
        "rag": stringify_rag(rag),
        "tokens": stringify_anom_tokens(toks),
    }

# Try to extract "Sequence ID" references from retrieved logs text
SEQ_ID_PATTERNS = [
    re.compile(r"\bSequence\s*ID\s*[:=]\s*(\d+)\b", re.IGNORECASE),
    re.compile(r"\bseq(?:uence)?[_\s-]*id\s*[:=]\s*(\d+)\b", re.IGNORECASE),
    re.compile(r"\bID\s*[:=]\s*(\d+)\b", re.IGNORECASE),
]

def parse_retrieved_seq_ids(rag_text: str) -> List[str]:
    if not rag_text:
        return []
    found: List[str] = []
    for pat in SEQ_ID_PATTERNS:
        for m in pat.finditer(rag_text):
            found.append(m.group(1))
    # de-dup while preserving order
    seen = set()
    out = []
    for x in found:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

# ----------------------------
# 4) Load JSON and build ID->classification map (for same-anomal matching)
# ----------------------------
with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

entries_raw = get_entries_container(raw_data)
assert isinstance(entries_raw, list), "Expected entries to be a list."

# Normalize all entries first
seqs_all: List[Dict[str, str]] = []
for e in entries_raw:
    if isinstance(e, dict):
        seqs_all.append(extract_sequence(e))
    else:
        # fallback
        seqs_all.append(extract_sequence({"text": str(e)}))

id_to_class: Dict[str, str] = {}
for s in seqs_all:
    # Keep first seen if duplicates exist
    if s["sequence_id"] not in id_to_class:
        id_to_class[s["sequence_id"]] = s["classification"]

# ----------------------------
# 5) Prompt template (format + requested additions)
# ----------------------------
SYSTEM_INSTRUCTIONS = (
    "You are an SRE incident analyst. Produce concise, operational incident analyses.\n"
    "Strict formatting rules:\n"
    "1) First line MUST be exactly: 'Sequence ID: <ID> | <Classification>'\n"
    "2) Then a blank line.\n"
    "3) Then 'Issue: <one-sentence summary of the current sequence only>'\n"
    "   - Do NOT mention other sequences in Issue.\n"
    "4) Then 'Moreover,': <single line explaining what the spikes imply (CPU/memory/duration/failed steps/network-io/gpu/error-rate/queue-depth/latency/failures/network-io/disk-io/gpu/latency/queue-depth/error-rate/retries/failures/thread-count/gc-pauses/others)>'\n"
   "5) Then 'Reason: <one paragraph>'\n"
"   - The first line must summarize the main cause of the anomaly in the current sequence in 10–20 words.\n"
"   - MUST include references(1-4,preferable 2-3) to relevant sequences and anomal sequences from the dataset (not only the previous sequence) if found, explaining how each one relates to the current sequence's anomal behavior.\n"
"   - Compare the current logs with retrieved historical logs(1-4,preferable 2-3) and explain which patterns (CPU/memory spikes, failed steps, network-io, GPU usage, error-rate, queue-depth, latency, thread-count, GC pauses, long durations, etc.) are shared.\n"
"   - For each relevant retrieved sequence (up to 2–4), provide a short explanation (10–30 words) describing why it is relevant and how its observed behavior can help explain the current sequence anomaly.\n"
"   - do not mention normal sequences and Explain why the historican anomal sequence is directly related to the current sequence anomaly, and clearly describe how its anomalous behavior contributes to or explains the anomaly in the current sequence.\n"
"   -  List each relevant retrieved sequence clearly, e.g., 'Sequence <ID>: <explanation>',(next line) 'Sequence <ID>: <explanation>'(next line) ,'Sequence <ID>: <explanation>'(next line) .......not exactly 3 sequence,use 1-5 sequences,preferable 2-3'.\n"
"   - If no relevant sequences are found, only provide the main Reason paragraph.\n"
"   - Do NOT mention sequences that are irrelevant or with a different classification.\n"
"6) Then 'Suggested fix:' followed by numbered remediation steps.\n"
"   - Format MUST be exactly:\n"
"     Suggested fix:\n"
"     1. <step one>\n"
"     2. <step two>\n"
"     3. <step three>\n"
"     4. <step four>\n"
"   - Use 3 to 5 numbered steps only.\n"
"   - Each step must be short one sentence and Keep it short and human-readable.\n"
"   - Do NOT write paragraphs.\n"
"   - Do NOT add blank lines between steps.\n"
"   - Do NOT use bullet points or hyphens.\n"
"   - Steps must be concrete operational actions.\n"
)

def build_user_prompt(seq: Dict[str, str],
                      prev_id: str,
                      prev_class: str,
                      same_class_ids_in_rag: List[str]) -> str:
    same_class_str = ", ".join(same_class_ids_in_rag) if same_class_ids_in_rag else "None detected"
    return (
        f"Sequence ID: {seq['sequence_id']}\n"
        f"Classification: {seq['classification']}\n"
        f"Prev sequence: {prev_id} | {prev_class}\n"
        f"Same-class retrieved Sequence IDs (if any): {same_class_str}\n\n"
        f"Logs:\n{seq['logs']}\n\n"
        f"Moreover,\n{seq['tokens']}\n\n"
        f"Retrieved similar historical logs (RAG):\n{seq['rag']}\n\n"
        "Now write the incident analysis using the required format."
    )

def format_as_chat(system_text: str, user_text: str) -> str:
    if hasattr(tokenizer, "apply_chat_template"):
        messages = [
            {"role": "system", "content": system_text},
            {"role": "user", "content": user_text},
        ]
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return f"<s>[SYSTEM]\n{system_text}\n[/SYSTEM]\n[USER]\n{user_text}\n[/USER]\n[ASSISTANT]\n"

def clean_output(text: str) -> str:
    t = text.strip()
    t = re.sub(r"</s>\s*$", "", t).strip()
    return t

def isolate_completion(decoded_text: str) -> str:
    t = clean_output(decoded_text)
    marker = "Now write the incident analysis using the required format."
    if marker in t:
        t = t.split(marker, 1)[-1].strip()
    # If model echoed multiple "Sequence ID:", keep last block
    if t.count("Sequence ID:") > 1:
        t = "Sequence ID:" + t.split("Sequence ID:")[-1].strip()
    return t

# ----------------------------
# 6) Batch generation loop
# ----------------------------
for start in range(0, len(seqs_all), BATCH_SIZE):
    batch = seqs_all[start:start + BATCH_SIZE]

    prompts: List[str] = []
    for i, seq in enumerate(batch):
        global_index = start + i

        # prev sequence (by file order)
        if global_index > 0:
            prev_seq = seqs_all[global_index - 1]
            prev_id, prev_class = prev_seq["sequence_id"], prev_seq["classification"]
        else:
            prev_id, prev_class = "None", "None"

        # find retrieved seq IDs inside RAG text (if present), then compute which are same classification
        retrieved_ids = parse_retrieved_seq_ids(seq["rag"])
        same_class_ids = []
        for rid in retrieved_ids:
            rclass = id_to_class.get(rid)
            if rclass and rclass == seq["classification"]:
                same_class_ids.append(rid)

        user_prompt = build_user_prompt(seq, prev_id, prev_class, same_class_ids)
        prompts.append(format_as_chat(SYSTEM_INSTRUCTIONS, user_prompt))

    enc = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_INPUT_LEN,
    ).to(model.device)

    with torch.inference_mode():
        out = model.generate(**enc, generation_config=gen_cfg)

    decoded = tokenizer.batch_decode(out, skip_special_tokens=True)

    # Print one-by-one immediately after each batch completes
    for d in decoded:
        print(isolate_completion(d))
        print("\n" + "-" * 80 + "\n")

print("Done.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[/INST] Sequence ID: 10 | Anomalous

Issue: A series of tests and deployments have shown increased duration and higher failure rates.

Moreover,
The CPU usage in the current sequence exhibits a significant increase compared to previous normal sequences. Specifically, the average CPU usage is 6.5901, which is 1.5 times higher than the average CPU usage in normal sequences (421.18). This increase in CPU usage may be contributing to the longer test and deployment durations observed in the current sequence.

Retrieved Sequence 5: test 15.21 3892.12 321.45 pass 0 0, with an average CPU usage of 321.45, similar to the current sequence's CPU usage. However, this sequence did not exhibit any failures or increased durations.

Retrieved Sequence 6: scan 22.17 5127.61 412.2 pass 0 1, with an average CPU usage of 412.2, also similar to the current sequence's CPU usage. This sequence had a single failure, but the failure rate was much lower than the current sequence.

Reason: The increased CPU usag

In [ ]:
!pip -q install fpdf

from fpdf import FPDF
import re

# ----------------------------
# 7) Prepare outputs for PDF
# ----------------------------

# Collect all isolated completions into a list
all_outputs = []

# If your loop already printed outputs, you can store them while printing:
# For example, after isolate_completion(d):
# all_outputs.append(isolate_completion(d))

# For demonstration, let's assume 'decoded' from last batch
for d in decoded:
    all_outputs.append(isolate_completion(d))

# ----------------------------
# 8) Create PDF
# ----------------------------

pdf = FPDF(orientation="P", unit="mm", format="A4")
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

for idx, text in enumerate(all_outputs, 1):
    pdf.multi_cell(0, 6, f"Sequence {idx}:\n{text}\n{'-'*80}\n")

# Save PDF
PDF_PATH = "/content/llm_incident_analysis.pdf"
pdf.output(PDF_PATH)

print(f"All sequence analyses saved to PDF: {PDF_PATH}")


All sequence analyses saved to PDF: /content/llm_incident_analysis.pdf
